In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown --id 1fPoZrntWDpIkSXJWQDZmHJK60FVcgQlb&export=download

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1fPoZrntWDpIkSXJWQDZmHJK60FVcgQlb
To: /content/model_step_50001.pt
100% 1.46G/1.46G [00:19<00:00, 73.4MB/s]


In [ ]:
!git clone https://github.com/nlpyang/BertSum.git

fatal: destination path 'BertSum' already exists and is not an empty directory.


In [ ]:
!pip install pytorch_pretrained_bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#importing dependencies from transformers
import torch
import pandas as pd
import difflib
from IPython.core.display import display, HTML
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from rouge import Rouge
#Set up the device for BertSum & Pegasus
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'google/pegasus-cnn_dailymail'

In [ ]:
def BertSum(model, clips, range=250, max = 3):
  generated_summaries = []
  for clip in clips:
    if len(generated_summaries) == range:
      break
    file_path = '/content/input.txt'
    with open(file_path, 'w') as file:
        file.write(clip)
    input_fp = '/content/input.txt'
    result_fp = '/content/results.txt'
    summary = summarize(input_fp, result_fp, model, max_length=max, device=device)
    generated_summaries += [summary]
  return generated_summaries

In [ ]:
def Pegasus(model, clips, ran=250, min = 50, max = 300):
  summary = []
  for i in range(ran):
    inputs = tokenizer(clips[i], truncation=True, max_length = 512, padding='longest', return_tensors='pt').to(device)

    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']


    translated = model.generate(input_ids=input_ids, attention_mask=attention_mask,min_length = min, max_length=max)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    summary += [tgt_text]
  return summary

In [ ]:
def Rougesh(summary, reference):
  rouge = Rouge()

  total_rouge_1_r = 0.0
  total_rouge_1_p = 0.0
  total_rouge_1_f = 0.0
  total_rouge_2_r = 0.0
  total_rouge_2_p = 0.0
  total_rouge_2_f = 0.0
  total_rouge_l_r = 0.0
  total_rouge_l_p = 0.0
  total_rouge_l_f = 0.0

  for i in range(len(summary)):


      generated_summary = summary[i]

      reference_summary = [reference[i]]

      scores = rouge.get_scores(generated_summary, reference_summary)

      total_rouge_1_r += scores[0]["rouge-1"]["r"]
      total_rouge_1_p += scores[0]["rouge-1"]["p"]
      total_rouge_1_f += scores[0]["rouge-1"]["f"]
      total_rouge_2_r += scores[0]["rouge-2"]["r"]
      total_rouge_2_p += scores[0]["rouge-2"]["p"]
      total_rouge_2_f += scores[0]["rouge-2"]["f"]
      total_rouge_l_r += scores[0]["rouge-l"]["r"]
      total_rouge_l_p += scores[0]["rouge-l"]["p"]
      total_rouge_l_f += scores[0]["rouge-l"]["f"]

  average_rouge_1_r = total_rouge_1_r / len(summary)
  average_rouge_1_p = total_rouge_1_p / len(summary)
  average_rouge_1_f = total_rouge_1_f / len(summary)
  average_rouge_2_r = total_rouge_2_r / len(summary)
  average_rouge_2_p = total_rouge_2_p / len(summary)
  average_rouge_2_f = total_rouge_2_f / len(summary)
  average_rouge_l_r = total_rouge_l_r / len(summary)
  average_rouge_l_p = total_rouge_l_p / len(summary)
  average_rouge_l_f = total_rouge_l_f / len(summary)

  print("Average ROUGE-1:")
  print(f"Average_R: {average_rouge_1_r:.5f}")
  print(f"Average_P: {average_rouge_1_p:.5f}")
  print(f"Average_F: {average_rouge_1_f:.5f}")
  print("---------------------------------------------")
  print("Average ROUGE-2:")
  print(f"Average_R: {average_rouge_2_r:.5f}")
  print(f"Average_P: {average_rouge_2_p:.5f}")
  print(f"Average_F: {average_rouge_2_f:.5f}")
  print("---------------------------------------------")
  print("Average ROUGE-L:")
  print(f"Average_R: {average_rouge_l_r:.5f}")
  print(f"Average_P: {average_rouge_l_p:.5f}")
  print(f"Average_F: {average_rouge_l_f:.5f}")
  print("---------------------------------------------")

In [ ]:
!gdown --id 14q1-GCaNnKfD9pcZu1oiFDuXOKqjlqTo -O module.py
!mv ./module.py /content/BertSum/src/models/

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=14q1-GCaNnKfD9pcZu1oiFDuXOKqjlqTo
To: /content/module.py
100% 4.43k/4.43k [00:00<00:00, 17.9MB/s]


In [ ]:
import sys
import textwrap
sys.path.append("/content/BertSum/src")

In [ ]:
!rm /content/BertSum/src/models/neural.py
!gdown 1z08OXhnlYre5SsvhMYzv8I1UmCcFlX-r -O neural.py
!mv ./neural.py /content/BertSum/src/models/

Downloading...
From: https://drive.google.com/uc?id=1z08OXhnlYre5SsvhMYzv8I1UmCcFlX-r
To: /content/neural.py
100% 8.14k/8.14k [00:00<00:00, 23.4MB/s]


In [ ]:
class Args:
    temp_dir = "temp"
    encoder = "transformer"
    max_pos = 512
    max_tgt_len = 140
    min_tgt_len = 55
    max_src_len = 510
    min_src_len = 5
    test_batch_size = 3000
    batch_size = 140
    ext_len = 0
    model_path = None
    visible_gpus = None
    gpu_ranks = None
    world_size = 1
    report_every = 50
    save_checkpoint_steps = 5
    recall_eval = False
    visible_device = None
    multi_gpu = False
    max_length = 200
    min_length = 50
    alpha = 0.95
    result_path = "results/cnndm"
    test_from = None
    test_start_from = None
    dataset = "cnndm"
    bert_config_path = "/content/BertSum/bert_config_uncased_base.json"
    trained_model_path = "/content/BertSum/models/bertsumext/model_step_50001.pt"
    param_init = 0.0
    param_init_glorot = True
    ff_size = 2048
    heads = 8
    dropout = 0.1
    inter_layers = 2
args = Args()

In [ ]:
def compare_texts(text1, text2, r = 0):
    # Split the texts into words
    words1 = text1.split()
    words2 = text2.split()

    # Create lists to hold the output words
    output1 = [''] * len(words1)
    output2 = [''] * len(words2)

    #Append the words to make possible bigrams for ROUGE-2
    if r == 1:
      bigram1 = [' '.join(words1[i:i+2]) for i in range(len(words1)-1)]
      bigram2 = [' '.join(words2[i:i+2]) for i in range(len(words2)-1)]
      matcher = difflib.SequenceMatcher(None, bigram1, bigram2)

    # Use difflib to compare the words & loop through the diff and add HTML tags for added and deleted words
    else:
      matcher = difflib.SequenceMatcher(None, words1, words2)

    if r == 0:
      diff = matcher.get_opcodes()
      for tag, i1, i2, j1, j2 in diff:
        if tag == 'equal':
            output1.extend(f'<span style="color: #00ff00">{word}</span>' for word in words1[i1:i2])
            output2.extend(f'<span style="color: #00ff00">{word}</span>' for word in words2[j1:j2])
        else:
            output1.extend(words1[i1:i2])
            output2.extend(words2[j1:j2])
    elif r == 1:
      diff = matcher.get_opcodes()
      for tag, i1, i2, j1, j2 in diff:
        if tag == 'equal':
            for i in range(i1, i2):
                if output1[i] != '':
                    output1[i] = output1[i].replace('color: #00ff00', 'color: #ff0000')
                else:
                    output1[i] = f'<span style="color: #00ff00">{words1[i]}</span>'

                output1[i+1] = f'<span style="color: #00ff00">{words1[i+1]}</span>'

            for j in range(j1, j2):
                if output2[j] != '':
                    output2[j] = output2[j].replace('color: #00ff00', 'color: #ff0000')
                    output2[j+1] = f'<span style="color: #00ff00">{words2[j+1]}</span>'
                else:
                    output2[j] = f'<span style="color: #00ff00">{words2[j]}</span>'
                    output2[j+1] = f'<span style="color: #00ff00">{words2[j+1]}</span>'

    # Join the words back into a single string
    elif r == 2:
      match = matcher.find_longest_match(0, len(words1), 0, len(words2))
      for i in range(len(words1)):
        if match.a <= i < match.a + match.size:
            output1.append(f'<span style="color: #00ff00">{words1[i]}</span>')
        else:
            output1.append(words1[i])

      for i in range(len(words2)):
        if match.b <= i < match.b + match.size:
            output2.append(f'<span style="color: #00ff00">{words2[i]}</span>')
        else:
            output2.append(words2[i])



    # Join the words back into a single string
    if r != 1:
      html1 = ' '.join(output1)
      html2 = ' '.join(output2)

    elif r == 1:
      html1 = ' '.join(word if word != '' else f'<span class="normal-text">{words1[i]}</span>' for i, word in enumerate(output1))
      html2 = ' '.join(word if word != '' else f'<span class="normal-text">{words2[i]}</span>' for i, word in enumerate(output2))

    # Create a table to display the texts side by side
    table = f'''
    <table>
        <tr>
            <th>Generated</th>
            <th>Reference</th>
        </tr>
        <tr>
            <td style="padding-right:20px">{html1}</td>
            <td>{html2}</td>
        </tr>
    </table>
    '''

    # Display the HTML
    display(HTML(table))

In [ ]:
import nltk
from models.model_builder import Summarizer
from models.module import summarize
from pytorch_pretrained_bert import BertConfig
nltk.download('punkt')
# Load the pre-trained BERT model
bert_config = BertConfig.from_json_file(args.bert_config_path)

# Load the BertSum model
model = Summarizer(args=args, device=device, load_pretrained_bert=False, bert_config=bert_config).to(device)
# Load the checkpoint
checkpoint = torch.load("/content/model_step_50001.pt", map_location=device)
model.load_cp(checkpoint)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!gdown --id 1uepQiNca9ZolZHB6HQeatLFiqsRyPEqX -O test.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1uepQiNca9ZolZHB6HQeatLFiqsRyPEqX
To: /content/test.zip
100% 18.9M/18.9M [00:00<00:00, 145MB/s]


In [ ]:
!unzip /content/test.zip

Archive:  /content/test.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
df = pd.read_csv("/content/test.csv")

In [ ]:
clips = df['article']
reference = df['highlights']

In [ ]:
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model_P = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

In [ ]:
#BertSum CNN/Dailymail
BertSum_Sum = BertSum(model,clips, range = 10, max = 3)
Rougesh([[bs] for bs in BertSum_Sum],reference)

Average ROUGE-1:
Average_R: 0.43213
Average_P: 0.31523
Average_F: 0.35879
---------------------------------------------
Average ROUGE-2:
Average_R: 0.18370
Average_P: 0.12582
Average_F: 0.14599
---------------------------------------------
Average ROUGE-L:
Average_R: 0.41519
Average_P: 0.30401
Average_F: 0.34532
---------------------------------------------


In [ ]:
#BertSum CNN/Dailymail
BertSum_Sum = BertSum(model,clips, range = 10, max = 3)
Rougesh([[bs] for bs in BertSum_Sum],reference)

Average ROUGE-1:
Average_R: 0.43213
Average_P: 0.31523
Average_F: 0.35879
---------------------------------------------
Average ROUGE-2:
Average_R: 0.18370
Average_P: 0.12582
Average_F: 0.14599
---------------------------------------------
Average ROUGE-L:
Average_R: 0.41519
Average_P: 0.30401
Average_F: 0.34532
---------------------------------------------


In [ ]:
#Pegasus
P_Sum = Pegasus(model_P,clips, ran = 2)
Rougesh(P_Sum,reference)

Average ROUGE-1:
Average_R: 0.49020
Average_P: 0.43353
Average_F: 0.45606
---------------------------------------------
Average ROUGE-2:
Average_R: 0.23030
Average_P: 0.20643
Average_F: 0.21642
---------------------------------------------
Average ROUGE-L:
Average_R: 0.49020
Average_P: 0.43353
Average_F: 0.45606
---------------------------------------------


In [ ]:
print(textwrap.fill(clips[1], width = 100))

A drunk teenage boy had to be rescued by security after jumping into a lions' enclosure at a zoo in
western India. Rahul Kumar, 17, clambered over the enclosure fence at the Kamla Nehru Zoological
Park in Ahmedabad, and began running towards the animals, shouting he would 'kill them'. Mr Kumar
explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators.
Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in
Ahmedabad and began running towards the animals shouting 'Today I kill a lion!' Mr Kumar had been
sitting near the enclosure when he suddenly made a dash for the lions, surprising zoo security. The
intoxicated teenager ran towards the lions, shouting: 'Today I kill a lion or a lion kills me!' A
zoo spokesman said: 'Guards had earlier spotted him close to the enclosure but had no idea he was
planing to enter it. 'Fortunately, there are eight moats to cross before getting to where the lions
usually are and he

<h1>ROUGE-1</h1>

In [ ]:
print("ROUGE-1")
print("BERTSum")
compare_texts(BertSum_Sum[1], reference[1], r = 0)

ROUGE-1
BERTSum


Generated,Reference
"Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in Ahmedabad and began running towards the animals shouting 'Today I kill a lion!' Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators. Mr Kumar had been sitting near the enclosure when he suddenly made a dash for the lions, surprising zoo security.","Drunk teenage boy climbed into lion enclosure at zoo in west India . Rahul Kumar, 17, ran towards animals shouting 'Today I kill a lion!' Fortunately he fell into a moat before reaching lions and was rescued ."


In [ ]:
print("PEGASUS")
compare_texts(P_Sum[1][0], reference[1], r = 0)

PEGASUS


Generated,Reference
"Rahul Kumar, 17, climbed into the enclosure at Kamla Nehru Zoological Park.He ran towards the animals shouting: 'Today I kill a lion or a lion kills me!'Fortunately, he fell into a moat and was rescued by zoo security.","Drunk teenage boy climbed into lion enclosure at zoo in west India . Rahul Kumar, 17, ran towards animals shouting 'Today I kill a lion!' Fortunately he fell into a moat before reaching lions and was rescued ."


<h1>ROUGE-2</h1>

In [ ]:
print("ROUGE-2")
print("BERTSum")
compare_texts(BertSum_Sum[1], reference[1], r = 1)

ROUGE-2
BERTSum


Generated,Reference
"Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in Ahmedabad and began running towards the animals shouting 'Today I kill a lion!' Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators. Mr Kumar had been sitting near the enclosure when he suddenly made a dash for the lions, surprising zoo security.","Drunk teenage boy climbed into lion enclosure at zoo in west India . Rahul Kumar, 17, ran towards animals shouting 'Today I kill a lion!' Fortunately he fell into a moat before reaching lions and was rescued ."


In [ ]:
print("PEGASUS")
compare_texts(P_Sum[1][0], reference[1], r = 1)

PEGASUS


Generated,Reference
"Rahul Kumar, 17, climbed into the enclosure at Kamla Nehru Zoological Park.He ran towards the animals shouting: 'Today I kill a lion or a lion kills me!'Fortunately, he fell into a moat and was rescued by zoo security.","Drunk teenage boy climbed into lion enclosure at zoo in west India . Rahul Kumar, 17, ran towards animals shouting 'Today I kill a lion!' Fortunately he fell into a moat before reaching lions and was rescued ."


<h1>ROUGE-L</h1>

In [ ]:
print("ROUGE-L")
print("BERTSum")
compare_texts(BertSum_Sum[1], reference[1], r = 2)

ROUGE-L
BERTSum


Generated,Reference
"Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in Ahmedabad and began running towards the animals shouting 'Today I kill a lion!' Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators. Mr Kumar had been sitting near the enclosure when he suddenly made a dash for the lions, surprising zoo security.","Drunk teenage boy climbed into lion enclosure at zoo in west India . Rahul Kumar, 17, ran towards animals shouting 'Today I kill a lion!' Fortunately he fell into a moat before reaching lions and was rescued ."


In [ ]:
print("PEGASUS")
compare_texts(P_Sum[1][0], reference[1], r = 2)

PEGASUS


Generated,Reference
"Rahul Kumar, 17, climbed into the enclosure at Kamla Nehru Zoological Park.He ran towards the animals shouting: 'Today I kill a lion or a lion kills me!'Fortunately, he fell into a moat and was rescued by zoo security.","Drunk teenage boy climbed into lion enclosure at zoo in west India . Rahul Kumar, 17, ran towards animals shouting 'Today I kill a lion!' Fortunately he fell into a moat before reaching lions and was rescued ."


In [ ]:
!gdown 19YWWVsDJk_KtArI-2Wi1HKU6ldviz32n -O gpt.csv

Downloading...
From: https://drive.google.com/uc?id=19YWWVsDJk_KtArI-2Wi1HKU6ldviz32n
To: /content/gpt.csv
100% 81.6k/81.6k [00:00<00:00, 71.4MB/s]


In [ ]:
data = pd.read_csv("/content/gpt.csv")

In [ ]:
clips = data['Clips']
reference = data['Reference']

In [ ]:
summary = BertSum(model,[clips[50]], max = 6)
Rougesh([summary],[reference[50]])

Average ROUGE-1:
Average_R: 0.61702
Average_P: 0.85294
Average_F: 0.71605
---------------------------------------------
Average ROUGE-2:
Average_R: 0.50000
Average_P: 0.75676
Average_F: 0.60215
---------------------------------------------
Average ROUGE-L:
Average_R: 0.51064
Average_P: 0.70588
Average_F: 0.59259
---------------------------------------------


In [ ]:
compare_texts(summary[0], reference[50], r = 0)

Generated,Reference
"With each dive, she unraveled the mysteries of the underwater world, documenting new species and advocating for their preservation. Beneath the turbulent waves of the ocean, Lia, a courageous marine biologist, explored the vibrant coral reefs teeming with life.","Lia, a courageous marine biologist, delved beneath the turbulent waves of the ocean, exploring vibrant coral reefs teeming with life. Through her dives, she unraveled the mysteries of the underwater world, documenting new species and advocating for their preservation. Lia's passion for marine conservation propelled her to protect the delicate balance of the ocean, revealing its beauty and advocating for its protection."


In [ ]:
compare_texts(summary[0], reference[50], r = 1)

Generated,Reference
"With each dive, she unraveled the mysteries of the underwater world, documenting new species and advocating for their preservation. Beneath the turbulent waves of the ocean, Lia, a courageous marine biologist, explored the vibrant coral reefs teeming with life.","Lia, a courageous marine biologist, delved beneath the turbulent waves of the ocean, exploring vibrant coral reefs teeming with life. Through her dives, she unraveled the mysteries of the underwater world, documenting new species and advocating for their preservation. Lia's passion for marine conservation propelled her to protect the delicate balance of the ocean, revealing its beauty and advocating for its protection."


In [ ]:
compare_texts(summary[0], reference[50], r = 2)

Generated,Reference
"With each dive, she unraveled the mysteries of the underwater world, documenting new species and advocating for their preservation. Beneath the turbulent waves of the ocean, Lia, a courageous marine biologist, explored the vibrant coral reefs teeming with life.","Lia, a courageous marine biologist, delved beneath the turbulent waves of the ocean, exploring vibrant coral reefs teeming with life. Through her dives, she unraveled the mysteries of the underwater world, documenting new species and advocating for their preservation. Lia's passion for marine conservation propelled her to protect the delicate balance of the ocean, revealing its beauty and advocating for its protection."


In [ ]:
#Pegasus GPT 3.5
P_Sum = Pegasus(model_P,clips,ran = len(clips))
Rougesh(P_Sum, reference)

Average ROUGE-1:
Average_R: 0.47032
Average_P: 0.54015
Average_F: 0.47664
---------------------------------------------
Average ROUGE-2:
Average_R: 0.29543
Average_P: 0.36953
Average_F: 0.30837
---------------------------------------------
Average ROUGE-L:
Average_R: 0.43073
Average_P: 0.50310
Average_F: 0.43991
---------------------------------------------


In [ ]:
#BertSum GPT 3.5
BertSum_Sum = BertSum(model,clips, leng = 6)
Rougesh([[x] for x in BertSum_Sum],reference)

Average ROUGE-1:
Average_R: 0.63973
Average_P: 0.52855
Average_F: 0.55449
---------------------------------------------
Average ROUGE-2:
Average_R: 0.42687
Average_P: 0.37153
Average_F: 0.37909
---------------------------------------------
Average ROUGE-L:
Average_R: 0.58047
Average_P: 0.48641
Average_F: 0.50706
---------------------------------------------
